# Neural Dimensionality Estimation

### Set-up + Imports

In [1]:
import setup
setup.main()
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

import neurometry.datasets.synthetic as synthetic
import numpy as np
import skdim

import matplotlib.pyplot as plt


import os

os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Working directory:  /Users/facosta/Desktop/code/neurometry/neurometry
Directory added to path:  /Users/facosta/Desktop/code/neurometry
Directory added to path:  /Users/facosta/Desktop/code/neurometry/neurometry


INFO:root:Using pytorch backend


In [45]:
num_points = 1000
intrinsic_dim = 2
num_neurons = 6

torus_points = synthetic.hypertorus(intrinsic_dim, num_points)
neural_torus, _ = synthetic.synthetic_neural_manifold(
    torus_points, num_neurons, "sigmoid", scales=gs.ones(num_neurons)
)

In [46]:
danco = skdim.id.DANCo().fit(neural_torus)
lpca = skdim.id.lPCA().fit_pw(neural_torus, n_neighbors=100, n_jobs=1)

In [47]:
print(
    f" Ground Truth: {intrinsic_dim}, Danco: {danco.dimension_}, LPCA: {np.mean(lpca.dimension_pw_)}"
)

 Ground Truth: 2, Danco: 6.0, LPCA: 5.919
